In [3]:
!git clone https://github.com/tmikolov/word2vec.git

Cloning into 'word2vec'...
remote: Enumerating objects: 148, done.
remote: Total 148 (delta 0), reused 0 (delta 0), pack-reused 148
Receiving objects: 100% (148/148), 119.14 KiB | 2.29 MiB/s, done.
Resolving deltas: 100% (86/86), done.


**Clone word2vec code from Git and execute make file**

In [14]:
%cd /content/word2vec

/content/word2vec


In [0]:
!make

**Word embeddings snippet for 'of'**

of -0.565323 0.236813 0.272062 -1.426221 0.428859 -1.386884 -0.589239 2.778557 -1.440811 0.791160 2.015024 0.394671 1.053778 3.489750

In [18]:
!wget -c http://nlp.cs.rpi.edu/course/spring19/enwiki.tar.gz

--2019-02-21 00:24:40--  http://nlp.cs.rpi.edu/course/spring19/enwiki.tar.gz
Resolving nlp.cs.rpi.edu (nlp.cs.rpi.edu)... 128.113.126.107
Connecting to nlp.cs.rpi.edu (nlp.cs.rpi.edu)|128.113.126.107|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 515364205 (491M) [application/x-gzip]
Saving to: ‘enwiki.tar.gz’

enwiki.tar.gz       100%[===================>] 491.49M  26.0MB/s    in 25s     

2019-02-21 00:25:05 (19.3 MB/s) - ‘enwiki.tar.gz’ saved [515364205/515364205]



In [0]:
!tar -xvzf enwiki.tar.gz

enwiki.sample.txt


In [0]:
!./word2vec -train enwiki.sample.txt -output emboutput_1.txt -size 50 -window 5 -negative 10 -cbow 1 -threads 10 -iter 5 -min-count 5

Starting training using file enwiki.sample.txt
Vocab size: 460155
Words in train file: 247257644
Alpha: 0.004003  Progress: 91.99%  Words/thread/sec: 235.24k  ^C


**Word2vec is trained to implement wiki english corpus on for 50 vector dimension**

In [0]:
!git clone https://github.com/glample/tagger.git

Cloning into 'tagger'...
remote: Enumerating objects: 61, done.
remote: Total 61 (delta 0), reused 0 (delta 0), pack-reused 61
Unpacking objects: 100% (61/61), done.


In [0]:
!python trainreduce.py

In [0]:
!wget -c http://nlp.cs.rpi.edu/course/spring19/name_tagging.tar.gz

In [0]:
!tar -xvzf name_tagging.tar.gz

The test file is used to get tokens without BIO tagging for usage in LSTM tagger

In [0]:
filename = open('/content/eng.test.clean.bio')
output = open('/content/testing.txt','w')
Final_list=[]
a=''
for i in filename:
  if i != '\n':
    #print i
    file = i.strip()
    file=file.split(' ')
    for j in range (0,len(file)):
      if j==0:
        a=a+file[j]+' '
  else:
    a=a.strip()
    #Final_list.append(a)
    output.write(a)
    print(a)
    a=''
    a='\n'
    output.write(a)
output.close()
filename.close()


Limit the train size to 10000 for LSTM tagger as epochs are fewer and that may compromise neural network iterations and weight assignment to each tag

In [0]:
filename = open('eng.train.clean.bio')
output = open('eng.train.clean.bio.txt','w')
Final_list=[]
a=''
for i in filename:
  if i == '\n':
    a=a.strip()
    c = c+1
    #Final_list.append(a)
    output.write(a)
    print(a)
    a=''
   if c == 10000:
    break

In [0]:
%cd tagger

/content/tagger


In [0]:
!cat /content/tagger/emboutput_1.txt

In [11]:
from google.colab import files
files.upload()

Saving emboutput_1.txt to emboutput_1.txt


In [0]:
!./train.py --train /content/eng.train.clean.bio.txt --dev /content/eng.dev.clean.bio --test /content/eng.test.clean.bio --char_dim 50 --word_dim 50 --pre_emb /content/tagger/emboutput_1.txt

In [37]:
!./tagger.py --model /content/tagger/models/tag_scheme=iobes,lower=False,zeros=False,char_dim=50,char_lstm_dim=25,char_bidirect=True,word_dim=50,word_lstm_dim=100,word_bidirect=True,pre_emb=emboutput_1.txt,all_emb=False,cap_dim=0,crf=True,dropout=0.5,lr_method=sgd-lr_.005  --input /content/testing.txt  --output /content/predicteng.txt

Loading model...
Compiling...
WARNING (theano.tensor.blas): We did not find a dynamic library in the library_dir of the library we use for blas. If you use ATLAS, make sure to compile it with dynamics library.
WARNING (theano.tensor.blas): We did not find a dynamic library in the library_dir of the library we use for blas. If you use ATLAS, make sure to compile it with dynamics library.
Tagging...
---- 1 lines tagged in 43.6801s ----


I have implemented dataframes to store tags, calculate tags based on 'O' and otherwise checks for true positive , false positive and false negatives.

Calculating precision and recall is actually quite easy. Imagine there are 100 positive cases among 10,000 cases. You want to predict which ones are positive, and you pick 200 to have a better chance of catching many of the 100 positive cases.  You record the IDs of your predictions, and when you get the actual results you sum up how many times you were right or wrong. There are four ways of being right or wrong: 

TN / True Negative: case was negative and predicted negative 
TP / True Positive: case was positive and predicted positive 
FN / False Negative: case was positive but predicted negative 
FP / False Positive: case was negative but predicted positive 

In [0]:
import pandas as pd
list1 =[]
list2=[]
set_s = set()
set_b = set()

with open('C:/Users/Yaminie/Downloads/yamini_testing_tagged.txt', 'r', encoding = "utf8") as file1:
    with open('C:/Users/Yaminie/Downloads/name_tagging/eng.test.clean.bio', 'r', encoding = "utf8") as file2:
      for line in file1:
          for word in line.split():
              #print(word)
              #print("-------------------------------")
              word = word.split("__")
              list1.append(word)
              set_s.add(word[1])
      print(set_s)
      df1 = pd.DataFrame(list1,columns =['Word','Tag'])
      print(df1)
      for l in file2:
          l=l.strip()
          l = l.split(" ")
          #print(l)
          if(l[0] != ''):
              list2.append(l)
              set_b.add(l[1])
      df2 = pd.DataFrame(list2,columns =['Word','Tag'])
      print(df2)
      print(set_b)

    tag1 = df1['Tag']
    tag2 = df2['Tag']
    true_positive = 0
    false_negative = 0
    false_positive = 0
    for i in range (0, len(tag1)):
          if tag1.loc[i] == tag2.loc[i] and tag1.loc[i]!='O':
              true_positive +=1
          elif tag1.loc[i]!='O' and tag2.loc[i] =='O':
              false_positive +=1
          elif tag1.loc[i] == 'O' and tag2.loc[i] != 'O':
              false_negative +=1
          elif tag1.loc[i]!='O' and tag2.loc[i] != 'O':
              false_negative +=1
              false_positive +=1
            
    precision = true_positive/(true_positive+false_positive)
    recall = true_positive/ (true_positive+false_negative)
    fscore = 2*(recall * precision) / (recall + precision)
    print(precision*100)
    print(recall*100)
    print(fscore)



**Upon executing script for precision recall , following were the results for English corpus:**


1.   Precision : 79.89977728285078
2.   Recall :       56.84669219595933
3.   F-score :    0.6643005940899622





** Performance and Error Analysis**

•	Worked: 
                  Successfully able to download and implement word2vec, used word vector dimension to 50.
                  It was convenient to use word2vec library for vector implementation which equate to one hot encoding.
                  Furthermore, LSTM  tagger was easy access and due to large size of english corpus, it was set to 10000 using a python script.
                  
                  
•	Improvements: BIO format chunks were not properly tagged, i.e. chunking in the form that 'Amazon Prime' is not regarded as two separate and taken as one. Observed that new lines caused confusion in tagging -- my tags were not implemeted properly without strip function. 

•	Training data used: 10000 of wiki dataset and complete dataset was used for English

•	Epochs ran: 16 for English, 20 for Spanish
 
Last epoch metrics:

accuracy:  97.53%; precision:  81.12%; recall:  76.84%; FB1:  78.92
              FAC: precision:  66.37%; recall:  69.44%; FB1:  67.87  113
              GPE: precision:  88.53%; recall:  89.62%; FB1:  89.07  2205
              LOC: precision:  53.60%; recall:  45.27%; FB1:  49.08  250
              ORG: precision:  68.47%; recall:  58.35%; FB1:  63.01  1367
              PER: precision:  87.69%; recall:  84.82%; FB1:  86.23  1446

Error Analysis:

1.  The size of corpus should be identified to implement it within runtime. 
2.   Tags were not easily identified without strip function.
3.   Further scope is chunking in a manner that recognition can be improved for Inside or Beginning.
4.   If one word is identified incorrectly in a sentence the others are as well.
5.   Test file using tagger was causing differences.

